# Binance Futures Testnet Trading Bot

A comprehensive trading bot for Binance Futures Testnet with support for:
- Market and Limit orders
- Advanced order types (Stop-Limit, OCO)
- Complete logging and error handling
- CLI interface for user interaction
- Real-time order tracking and management

## Prerequisites
1. Register at Binance Testnet: https://testnet.binancefuture.com
2. Generate API credentials (API Key and Secret Key)
3. Install required packages: `pip install python-binance colorama python-decouple`

## Important Notes
-  This bot uses **TESTNET** only - no real money involved
-  Never share your API credentials
-  Always test thoroughly before any live trading
-  Use proper risk management

In [74]:
# Install required packages (run this cell first if packages are not installed)
import subprocess
import sys

def install_package(package):
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"Successfully installed {package}")
    except subprocess.CalledProcessError:
        print(f"Failed to install {package}")

# List of required packages
packages = [
    "python-binance==1.0.19",
    "colorama==0.4.6",
    "python-decouple==3.8"
]

print("Installing required packages...")
for package in packages:
    install_package(package)
print("Installation complete!")

Installing required packages...
Installation complete!


In [75]:
# Import required libraries
import logging
import time
import json
from datetime import datetime
from typing import Dict, Any, Optional, List
from decimal import Decimal
import os

# Third-party imports
from binance.client import Client
from binance.enums import *
from binance.exceptions import BinanceAPIException, BinanceOrderException
from colorama import init, Fore, Back, Style

# Initialize colorama for colored terminal output
init(autoreset=True)

print(f"{Fore.GREEN} All libraries imported successfully!")

 All libraries imported successfully!


In [ ]:
# Configuration and Logging Setup
class Config:
    
    # Binance Testnet URLs
    TESTNET_BASE_URL = "https://testnet.binancefuture.com"
    TESTNET_API_URL = "https://testnet.binancefuture.com/fapi/v1"
    
    # Default trading parameters
    DEFAULT_SYMBOL = "BTCUSDT"
    DEFAULT_QUANTITY = 0.001
    
    # Risk management
    MAX_POSITION_SIZE = 0.1
    MAX_DAILY_TRADES = 50

def setup_logging():
    
    # Create logs directory if it doesn't exist
    if not os.path.exists('logs'):
        os.makedirs('logs')
    
    # Configure logging with UTF-8 encoding to handle Unicode characters
    log_filename = f"logs/trading_bot_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"
    
    # Create file handler with UTF-8 encoding
    file_handler = logging.FileHandler(log_filename, encoding='utf-8')
    file_handler.setLevel(logging.INFO)
    
    # Create console handler
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)
    
    # Create formatter (without emoji characters to avoid encoding issues)
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    file_handler.setFormatter(formatter)
    console_handler.setFormatter(formatter)
    
    # Configure root logger
    logging.basicConfig(
        level=logging.INFO,
        handlers=[file_handler, console_handler]
    )
    
    logger = logging.getLogger('TradingBot')
    logger.info("Trading bot logging initialized")
    return logger

# Initialize logger
logger = setup_logging()
print(f"{Fore.GREEN} Logging setup complete!")

2025-10-02 23:41:42,405 - TradingBot - INFO - Trading bot logging initialized


 Logging setup complete!


In [ ]:
# Main Trading Bot Class
class BinanceTradingBot:
    
    def __init__(self, api_key: str, api_secret: str, testnet: bool = True):

        self.api_key = api_key
        self.api_secret = api_secret
        self.testnet = testnet
        self.logger = logging.getLogger('TradingBot')
        
        # Initialize Binance client
        try:
            self.client = Client(
                api_key=api_key,
                api_secret=api_secret,
                testnet=testnet
            )
            
            # Test connection
            self.client.futures_ping()
            self.logger.info("Successfully connected to Binance Futures Testnet")
            
        except Exception as e:
            self.logger.error(f"Failed to connect to Binance: {str(e)}")
            raise
    
    def get_account_info(self) -> Dict[str, Any]:

        try:
            account_info = self.client.futures_account()
            balance = self.client.futures_account_balance()
            
            self.logger.info("Account information retrieved successfully")
            return {
                'account_info': account_info,
                'balance': balance
            }
        except BinanceAPIException as e:
            self.logger.error(f"API Error getting account info: {e}")
            raise
        except Exception as e:
            self.logger.error(f"Unexpected error: {e}")
            raise
    
    def get_symbol_info(self, symbol: str) -> Dict[str, Any]:

        try:
            # Get symbol info
            exchange_info = self.client.futures_exchange_info()
            symbol_info = None
            
            for s in exchange_info['symbols']:
                if s['symbol'] == symbol:
                    symbol_info = s
                    break
            
            if not symbol_info:
                raise ValueError(f"Symbol {symbol} not found")
            
            # Get current price
            ticker = self.client.futures_symbol_ticker(symbol=symbol)
            
            self.logger.info(f"Retrieved info for {symbol}")
            return {
                'symbol_info': symbol_info,
                'current_price': float(ticker['price'])
            }
        except Exception as e:
            self.logger.error(f"Error getting symbol info: {e}")
            raise

# Test the basic bot initialization (you'll need to add your API credentials)
print(f"{Fore.CYAN} Trading Bot class defined successfully!")
print(f"{Fore.YELLOW} Remember to add your API credentials in the next cell!")

 Trading Bot class defined successfully!
 Remember to add your API credentials in the next cell!


In [ ]:
# API Credentials Setup (Clean Version)
# Your Binance Testnet API credentials
# Get them from: https://testnet.binancefuture.com

import threading
import time

# Your Testnet API Credentials (SAFE - TESTNET ONLY)
API_KEY = "213ca8b205bc77305ea2d12332ecaa0cf1989bb30d05b9a9437b49be3badb6cc"
API_SECRET = "4a541e079132d28e15f4f0a600de2b89b7bdd8b5e964315e2d5e7429b4dfc23c"

print("API credentials are set!")
print("Using TESTNET environment - No real money involved!")

# Function to initialize bot with timeout
def init_bot_with_timeout():

    global bot, init_result, init_error
    try:
        bot = BinanceTradingBot(API_KEY, API_SECRET, testnet=True)
        init_result = "success"
    except Exception as e:
        init_error = str(e)
        init_result = "error"
        bot = None

# Initialize variables
bot = None
init_result = None
init_error = None

print("Initializing bot connection (max 15 seconds)...")

# Start initialization in a separate thread
init_thread = threading.Thread(target=init_bot_with_timeout)
init_thread.daemon = True
init_thread.start()

# Wait for initialization with timeout
timeout_seconds = 15
for i in range(timeout_seconds):
    if init_result is not None:
        break
    time.sleep(1)
    if i % 3 == 0:  # Print dots every 3 seconds
        print(f"Connecting... ({i+1}s)")

# Check results
if init_result == "success":
    print("Trading bot initialized successfully!")
    print("Connected to Binance Futures TESTNET")
    print("You'll receive virtual USDT for practice trading")
    
elif init_result == "error":
    print(f"Failed to initialize bot: {init_error}")
    if "API" in init_error:
        print("Check your API credentials at https://testnet.binancefuture.com")
    else:
        print("This might be a network or server issue. Try again in a moment.")
    bot = None
    
else:
    print(f"Connection timed out after {timeout_seconds} seconds!")
    print("Possible causes:")
    print("   • Network connectivity issues")
    print("   • Binance testnet server maintenance") 
    print("   • Firewall blocking the connection")
    print("Try running this cell again in a few moments")
    bot = None

# Final status
if bot is not None:
    print("Bot is ready for trading!")
else:
    print("Bot not initialized - you can try again later")

API credentials are set!
Using TESTNET environment - No real money involved!
Initializing bot connection (max 15 seconds)...
Connecting... (1s)


2025-10-02 23:41:43,594 - TradingBot - INFO - Successfully connected to Binance Futures Testnet


Trading bot initialized successfully!
Connected to Binance Futures TESTNET
You'll receive virtual USDT for practice trading
Bot is ready for trading!


In [ ]:
# Order Management Methods - Part 1
# Add these methods to the BinanceTradingBot class

def add_order_methods_to_bot():
    
    def place_market_order(self, symbol: str, side: str, quantity: float, 
                          reduce_only: bool = False) -> Dict[str, Any]:
        try:
            self.logger.info(f"Placing market order: {side} {quantity} {symbol}")
            
            order = self.client.futures_create_order(
                symbol=symbol,
                side=side,
                type=ORDER_TYPE_MARKET,
                quantity=quantity,
                reduceOnly=reduce_only
            )
            
            self.logger.info(f"Market order placed successfully: {order['orderId']}")
            return order
            
        except BinanceOrderException as e:
            self.logger.error(f"Order error: {e}")
            raise
        except BinanceAPIException as e:
            self.logger.error(f"API error: {e}")
            raise
        except Exception as e:
            self.logger.error(f"Unexpected error: {e}")
            raise
    
    def place_limit_order(self, symbol: str, side: str, quantity: float, 
                         price: float, time_in_force: str = TIME_IN_FORCE_GTC,
                         reduce_only: bool = False) -> Dict[str, Any]:
        
        try:
            self.logger.info(f"Placing limit order: {side} {quantity} {symbol} @ {price}")
            
            order = self.client.futures_create_order(
                symbol=symbol,
                side=side,
                type=ORDER_TYPE_LIMIT,
                timeInForce=time_in_force,
                quantity=quantity,
                price=price,
                reduceOnly=reduce_only
            )
            
            self.logger.info(f"Limit order placed successfully: {order['orderId']}")
            return order
            
        except Exception as e:
            self.logger.error(f"Error placing limit order: {e}")
            raise
    
    # Add methods to the class
    BinanceTradingBot.place_market_order = place_market_order
    BinanceTradingBot.place_limit_order = place_limit_order

# Call the function to add methods
add_order_methods_to_bot()
print("Order management methods added successfully!")

Order management methods added successfully!


In [ ]:
# Advanced Order Types - Stop-Limit and OCO
def add_advanced_order_methods():
    
    def place_stop_limit_order(self, symbol: str, side: str, quantity: float,
                              stop_price: float, limit_price: float,
                              time_in_force: str = TIME_IN_FORCE_GTC,
                              reduce_only: bool = False) -> Dict[str, Any]:
        
        try:
            self.logger.info(f"Placing stop-limit order: {side} {quantity} {symbol}")
            self.logger.info(f"   Stop: {stop_price}, Limit: {limit_price}")
            
            order = self.client.futures_create_order(
                symbol=symbol,
                side=side,
                type=FUTURE_ORDER_TYPE_STOP,
                timeInForce=time_in_force,
                quantity=quantity,
                price=limit_price,
                stopPrice=stop_price,
                reduceOnly=reduce_only
            )
            
            self.logger.info(f"Stop-limit order placed: {order['orderId']}")
            return order
            
        except Exception as e:
            self.logger.error(f"Error placing stop-limit order: {e}")
            raise
    
    def place_oco_order(self, symbol: str, side: str, quantity: float,
                       price: float, stop_price: float, stop_limit_price: float,
                       time_in_force: str = TIME_IN_FORCE_GTC) -> Dict[str, Any]:
        
        try:
            self.logger.info(f"Placing OCO order: {side} {quantity} {symbol}")
            
            # For futures, we simulate OCO by placing two separate orders
            # This is a simplified implementation
            orders = []
            
            # Place limit order
            limit_order = self.place_limit_order(
                symbol=symbol,
                side=side,
                quantity=quantity,
                price=price,
                time_in_force=time_in_force
            )
            orders.append(limit_order)
            
            # Place stop-limit order
            opposite_side = SIDE_SELL if side == SIDE_BUY else SIDE_BUY
            stop_order = self.place_stop_limit_order(
                symbol=symbol,
                side=opposite_side,
                quantity=quantity,
                stop_price=stop_price,
                limit_price=stop_limit_price,
                time_in_force=time_in_force,
                reduce_only=True
            )
            orders.append(stop_order)
            
            self.logger.info(f"OCO orders placed successfully")
            return {'orders': orders, 'type': 'OCO_SIMULATION'}
            
        except Exception as e:
            self.logger.error(f"Error placing OCO order: {e}")
            raise
    
    def cancel_order(self, symbol: str, order_id: int) -> Dict[str, Any]:

        try:
            self.logger.info(f"Cancelling order {order_id} for {symbol}")
            
            result = self.client.futures_cancel_order(
                symbol=symbol,
                orderId=order_id
            )
            
            self.logger.info(f"Order cancelled successfully")
            return result
            
        except Exception as e:
            self.logger.error(f"Error cancelling order: {e}")
            raise
    
    def get_order_status(self, symbol: str, order_id: int) -> Dict[str, Any]:

        try:
            order = self.client.futures_get_order(
                symbol=symbol,
                orderId=order_id
            )
            
            self.logger.info(f"Retrieved order status for {order_id}")
            return order
            
        except Exception as e:
            self.logger.error(f"Error getting order status: {e}")
            raise
    
    # Add methods to the class
    BinanceTradingBot.place_stop_limit_order = place_stop_limit_order
    BinanceTradingBot.place_oco_order = place_oco_order
    BinanceTradingBot.cancel_order = cancel_order
    BinanceTradingBot.get_order_status = get_order_status

# Execute the function to add advanced methods
add_advanced_order_methods()
print(f"{Fore.GREEN}Advanced order methods (Stop-Limit, OCO) added to bot!")

Advanced order methods (Stop-Limit, OCO) added to bot!


In [ ]:
# Position and Portfolio Management
def add_position_management_methods():
    
    def get_positions(self, symbol: str = None) -> List[Dict[str, Any]]:

        try:
            positions = self.client.futures_position_information(symbol=symbol)
            
            # Filter only positions with size > 0
            active_positions = [
                pos for pos in positions 
                if float(pos['positionAmt']) != 0
            ]
            
            self.logger.info(f"Retrieved {len(active_positions)} active positions")
            return active_positions
            
        except Exception as e:
            self.logger.error(f"Error getting positions: {e}")
            raise
    
    def get_open_orders(self, symbol: str = None) -> List[Dict[str, Any]]:

        try:
            orders = self.client.futures_get_open_orders(symbol=symbol)
            self.logger.info(f"Retrieved {len(orders)} open orders")
            return orders
            
        except Exception as e:
            self.logger.error(f"Error getting open orders: {e}")
            raise
    
    def close_position(self, symbol: str, percentage: float = 100.0) -> Dict[str, Any]:
        
        try:
            positions = self.get_positions(symbol=symbol)
            if not positions:
                raise ValueError(f"No position found for {symbol}")
            
            position = positions[0]
            position_amt = float(position['positionAmt'])
            
            if position_amt == 0:
                raise ValueError(f"No position to close for {symbol}")
            
            # Calculate quantity to close
            close_quantity = abs(position_amt) * (percentage / 100.0)
            
            # Determine side (opposite of current position)
            side = SIDE_BUY if position_amt < 0 else SIDE_SELL
            
            self.logger.info(f"Closing {percentage}% of {symbol} position")
            
            # Place market order to close
            order = self.place_market_order(
                symbol=symbol,
                side=side,
                quantity=close_quantity,
                reduce_only=True
            )
            
            return order
            
        except Exception as e:
            self.logger.error(f"Error closing position: {e}")
            raise
    
    def cancel_all_orders(self, symbol: str = None) -> List[Dict[str, Any]]:

        try:
            if symbol:
                result = self.client.futures_cancel_all_open_orders(symbol=symbol)
                self.logger.info(f"Cancelled all orders for {symbol}")
            else:
                # Get all symbols with open orders and cancel them
                open_orders = self.get_open_orders()
                symbols = list(set([order['symbol'] for order in open_orders]))
                result = []
                
                for sym in symbols:
                    cancel_result = self.client.futures_cancel_all_open_orders(symbol=sym)
                    result.extend(cancel_result)
                
                self.logger.info(f"Cancelled all orders for {len(symbols)} symbols")
            
            return result
            
        except Exception as e:
            self.logger.error(f"Error cancelling orders: {e}")
            raise
    
    # Add methods to the class
    BinanceTradingBot.get_positions = get_positions
    BinanceTradingBot.get_open_orders = get_open_orders
    BinanceTradingBot.close_position = close_position
    BinanceTradingBot.cancel_all_orders = cancel_all_orders

# Execute function to add position methods
add_position_management_methods()
print(f"{Fore.GREEN}Position and portfolio management methods added!")

Position and portfolio management methods added!


In [ ]:
# Command Line Interface (CLI)
class TradingBotCLI:
    
    def __init__(self, bot: BinanceTradingBot):
        self.bot = bot
        self.logger = logging.getLogger('TradingBotCLI')
    
    def print_banner(self):

        banner = f"""
{'='*60}
BINANCE FUTURES TESTNET TRADING BOT
{'='*60}
Connected to Binance Testnet
This is TESTNET - No real money involved
{'='*60}
        """
        print(banner)
    
    def print_menu(self):

        menu = f"""
MAIN MENU:
1.  Account Information
2.  Symbol Information
3.  Place Market Order
4.  Place Limit Order
5.  Place Stop-Limit Order
6.  Place OCO Order
7.  View Positions
8.  View Open Orders
9.  Cancel Order
10. Cancel All Orders
11. Close Position
12. Account Balance
0.  Exit
        """
        print(menu)
    
    def get_user_input(self, prompt: str, input_type: type = str):

        while True:
            try:
                user_input = input(f"{prompt}: ")
                if input_type == str:
                    return user_input.strip().upper() if user_input.strip() else None
                elif input_type == float:
                    return float(user_input) if user_input.strip() else None
                elif input_type == int:
                    return int(user_input) if user_input.strip() else None
            except ValueError:
                print("Invalid input. Please try again.")
            except KeyboardInterrupt:
                print("\nGoodbye!")
                return None
    
    def display_account_info(self):

        try:
            account_data = self.bot.get_account_info()
            account_info = account_data['account_info']
            balance = account_data['balance']
            
            print("\nACCOUNT INFORMATION:")
            print(f"Total Wallet Balance: {account_info['totalWalletBalance']} USDT")
            print(f"Available Balance: {account_info['availableBalance']} USDT")
            print(f"Total Unrealized PnL: {account_info['totalUnrealizedProfit']} USDT")
            
            print("\nBALANCE DETAILS:")
            for bal in balance:
                if float(bal['balance']) > 0:
                    print(f"{bal['asset']}: {bal['balance']}")
                    
        except Exception as e:
            print(f"Error: {e}")
    
    def display_symbol_info(self):

        symbol = self.get_user_input("Enter symbol (e.g., BTCUSDT)", str)
        if not symbol:
            return
            
        try:
            info = self.bot.get_symbol_info(symbol)
            
            print(f"\nSYMBOL INFORMATION:")
            print(f"Symbol: {symbol}")
            print(f"Current Price: {info['current_price']} USDT")
            print(f"Status: {info['symbol_info']['status']}")
            
        except Exception as e:
            print(f"Error: {e}")

# Create CLI instance if bot is available
if 'bot' in globals() and bot is not None:
    cli = TradingBotCLI(bot)
    print("CLI interface created successfully!")
else:
    print("Please initialize the bot first (set API credentials)")

CLI interface created successfully!


In [ ]:
# CLI Trading Methods
def add_cli_trading_methods():
    
    def place_market_order_cli(self):

        print(f"\n{Fore.CYAN} PLACE MARKET ORDER:")
        
        symbol = self.get_user_input("Symbol (e.g., BTCUSDT)", str)
        if not symbol: return
        
        side = self.get_user_input("Side (BUY/SELL)", str)
        if side not in ['BUY', 'SELL']: 
            print(f"{Fore.RED} Invalid side. Must be BUY or SELL")
            return
        
        quantity = self.get_user_input("Quantity", float)
        if not quantity or quantity <= 0:
            print(f"{Fore.RED} Invalid quantity")
            return
        
        try:
            order = self.bot.place_market_order(symbol, side, quantity)
            print(f"{Fore.GREEN} Market order placed!")
            print(f"Order ID: {order['orderId']}")
            print(f"Status: {order['status']}")
            
        except Exception as e:
            print(f"{Fore.RED} Error: {e}")
    
    def place_limit_order_cli(self):

        print(f"\n{Fore.CYAN} PLACE LIMIT ORDER:")
        
        symbol = self.get_user_input("Symbol (e.g., BTCUSDT)", str)
        if not symbol: return
        
        side = self.get_user_input("Side (BUY/SELL)", str)
        if side not in ['BUY', 'SELL']: 
            print(f"{Fore.RED} Invalid side. Must be BUY or SELL")
            return
        
        quantity = self.get_user_input("Quantity", float)
        if not quantity or quantity <= 0:
            print(f"{Fore.RED} Invalid quantity")
            return
        
        price = self.get_user_input("Limit Price", float)
        if not price or price <= 0:
            print(f"{Fore.RED} Invalid price")
            return
        
        try:
            order = self.bot.place_limit_order(symbol, side, quantity, price)
            print(f"{Fore.GREEN} Limit order placed!")
            print(f"Order ID: {order['orderId']}")
            print(f"Status: {order['status']}")
            
        except Exception as e:
            print(f"{Fore.RED} Error: {e}")
    
    def place_stop_limit_order_cli(self):

        print(f"\n{Fore.CYAN} PLACE STOP-LIMIT ORDER:")
        
        symbol = self.get_user_input("Symbol (e.g., BTCUSDT)", str)
        if not symbol: return
        
        side = self.get_user_input("Side (BUY/SELL)", str)
        if side not in ['BUY', 'SELL']: 
            print(f"{Fore.RED} Invalid side. Must be BUY or SELL")
            return
        
        quantity = self.get_user_input("Quantity", float)
        if not quantity or quantity <= 0:
            print(f"{Fore.RED} Invalid quantity")
            return
        
        stop_price = self.get_user_input("Stop Price", float)
        if not stop_price or stop_price <= 0:
            print(f"{Fore.RED} Invalid stop price")
            return
        
        limit_price = self.get_user_input("Limit Price", float)
        if not limit_price or limit_price <= 0:
            print(f"{Fore.RED} Invalid limit price")
            return
        
        try:
            order = self.bot.place_stop_limit_order(symbol, side, quantity, stop_price, limit_price)
            print(f"{Fore.GREEN} Stop-limit order placed!")
            print(f"Order ID: {order['orderId']}")
            print(f"Status: {order['status']}")
            
        except Exception as e:
            print(f"{Fore.RED} Error: {e}")
    
    def view_positions_cli(self):

        try:
            positions = self.bot.get_positions()
            
            if not positions:
                print(f"{Fore.YELLOW} No active positions found")
                return
            
            print(f"\n{Fore.GREEN} ACTIVE POSITIONS:")
            for pos in positions:
                symbol = pos['symbol']
                size = float(pos['positionAmt'])
                entry_price = float(pos['entryPrice'])
                pnl = float(pos['unRealizedProfit'])
                
                direction = "LONG" if size > 0 else "SHORT"
                color = Fore.GREEN if pnl >= 0 else Fore.RED
                
                print(f"{Fore.CYAN}{symbol}: {direction} {abs(size)} @ {entry_price}")
                print(f"  {color}PnL: {pnl} USDT")
                
        except Exception as e:
            print(f"{Fore.RED} Error: {e}")
    
    def view_open_orders_cli(self):

        try:
            orders = self.bot.get_open_orders()
            
            if not orders:
                print(f"{Fore.YELLOW} No open orders found")
                return
            
            print(f"\n{Fore.GREEN} OPEN ORDERS:")
            for order in orders:
                print(f"ID: {order['orderId']} | {order['symbol']} | {order['side']} | {order['type']}")
                print(f"  Qty: {order['origQty']} @ {order.get('price', 'Market')}")
                print(f"  Status: {order['status']}")
                print("-" * 40)
                
        except Exception as e:
            print(f"{Fore.RED} Error: {e}")
    
    # Add methods to the CLI class
    TradingBotCLI.place_market_order_cli = place_market_order_cli
    TradingBotCLI.place_limit_order_cli = place_limit_order_cli
    TradingBotCLI.place_stop_limit_order_cli = place_stop_limit_order_cli
    TradingBotCLI.view_positions_cli = view_positions_cli
    TradingBotCLI.view_open_orders_cli = view_open_orders_cli

# Execute the function
add_cli_trading_methods()
print(f"{Fore.GREEN} CLI trading methods added successfully!")

 CLI trading methods added successfully!


In [ ]:
# Main CLI Runner - Interactive Trading Interface
def add_main_cli_runner():
    
    def run_cli(self):

        self.print_banner()
        
        while True:
            try:
                self.print_menu()
                choice = self.get_user_input("Select option (0-12)", str)
                
                if choice == "0":
                    print(f"{Fore.YELLOW} Goodbye! Happy trading!")
                    break
                elif choice == "1":
                    self.display_account_info()
                elif choice == "2":
                    self.display_symbol_info()
                elif choice == "3":
                    self.place_market_order_cli()
                elif choice == "4":
                    self.place_limit_order_cli()
                elif choice == "5":
                    self.place_stop_limit_order_cli()
                elif choice == "6":
                    self.place_oco_order_cli()
                elif choice == "7":
                    self.view_positions_cli()
                elif choice == "8":
                    self.view_open_orders_cli()
                elif choice == "9":
                    self.cancel_order_cli()
                elif choice == "10":
                    self.cancel_all_orders_cli()
                elif choice == "11":
                    self.close_position_cli()
                elif choice == "12":
                    self.display_account_info()
                else:
                    print(f"{Fore.RED} Invalid option. Please try again.")
                
                input(f"\n{Fore.CYAN}Press Enter to continue...")
                
            except KeyboardInterrupt:
                print(f"{Fore.YELLOW}\n Goodbye!")
                break
            except Exception as e:
                print(f"{Fore.RED} Unexpected error: {e}")
                input(f"\n{Fore.CYAN}Press Enter to continue...")
    
    def cancel_order_cli(self):

        print(f"\n{Fore.CYAN} CANCEL ORDER:")
        
        # First show open orders
        self.view_open_orders_cli()
        
        symbol = self.get_user_input("Symbol", str)
        if not symbol: return
        
        order_id = self.get_user_input("Order ID", int)
        if not order_id: return
        
        try:
            result = self.bot.cancel_order(symbol, order_id)
            print(f"{Fore.GREEN} Order {order_id} cancelled successfully!")
            
        except Exception as e:
            print(f"{Fore.RED} Error: {e}")
    
    def cancel_all_orders_cli(self):

        print(f"\n{Fore.CYAN} CANCEL ALL ORDERS:")
        
        symbol = self.get_user_input("Symbol (leave empty for all symbols)", str)
        
        confirm = self.get_user_input("Are you sure? (YES/NO)", str)
        if confirm != "YES":
            print(f"{Fore.YELLOW}Operation cancelled")
            return
        
        try:
            result = self.bot.cancel_all_orders(symbol if symbol else None)
            print(f"{Fore.GREEN} All orders cancelled successfully!")
            
        except Exception as e:
            print(f"{Fore.RED} Error: {e}")
    
    def close_position_cli(self):

        print(f"\n{Fore.CYAN} CLOSE POSITION:")
        
        # First show positions
        self.view_positions_cli()
        
        symbol = self.get_user_input("Symbol", str)
        if not symbol: return
        
        percentage = self.get_user_input("Percentage to close (1-100)", float)
        if not percentage or percentage <= 0 or percentage > 100:
            print(f"{Fore.RED} Invalid percentage")
            return
        
        try:
            order = self.bot.close_position(symbol, percentage)
            print(f"{Fore.GREEN} Position closed successfully!")
            print(f"Order ID: {order['orderId']}")
            
        except Exception as e:
            print(f"{Fore.RED} Error: {e}")
    
    def place_oco_order_cli(self):

        print(f"\n{Fore.CYAN} PLACE OCO ORDER:")
        print(f"{Fore.YELLOW}Note: This creates separate limit and stop-limit orders")
        
        symbol = self.get_user_input("Symbol (e.g., BTCUSDT)", str)
        if not symbol: return
        
        side = self.get_user_input("Side (BUY/SELL)", str)
        if side not in ['BUY', 'SELL']: 
            print(f"{Fore.RED} Invalid side. Must be BUY or SELL")
            return
        
        quantity = self.get_user_input("Quantity", float)
        if not quantity or quantity <= 0:
            print(f"{Fore.RED} Invalid quantity")
            return
        
        limit_price = self.get_user_input("Limit Price", float)
        if not limit_price or limit_price <= 0:
            print(f"{Fore.RED} Invalid limit price")
            return
        
        stop_price = self.get_user_input("Stop Price", float)
        if not stop_price or stop_price <= 0:
            print(f"{Fore.RED} Invalid stop price")
            return
        
        stop_limit_price = self.get_user_input("Stop Limit Price", float)
        if not stop_limit_price or stop_limit_price <= 0:
            print(f"{Fore.RED} Invalid stop limit price")
            return
        
        try:
            result = self.bot.place_oco_order(symbol, side, quantity, limit_price, 
                                            stop_price, stop_limit_price)
            print(f"{Fore.GREEN} OCO orders placed successfully!")
            for order in result['orders']:
                print(f"Order ID: {order['orderId']}")
            
        except Exception as e:
            print(f"{Fore.RED} Error: {e}")
    
    # Add methods to the CLI class
    TradingBotCLI.run_cli = run_cli
    TradingBotCLI.cancel_order_cli = cancel_order_cli
    TradingBotCLI.cancel_all_orders_cli = cancel_all_orders_cli
    TradingBotCLI.close_position_cli = close_position_cli
    TradingBotCLI.place_oco_order_cli = place_oco_order_cli

# Execute the function
add_main_cli_runner()
print(f"{Fore.GREEN} Main CLI runner added successfully!")

 Main CLI runner added successfully!


## Testing & Examples

Here are some example code snippets to test individual bot functions without the CLI:

In [90]:
# START THE TRADING BOT CLI
# Run this cell to start the interactive trading interface

if 'bot' in globals() and bot is not None and 'cli' in globals():
    print(f"{Fore.GREEN}Starting the Binance Trading Bot CLI...")
    print(f"{Fore.YELLOW}Make sure you have set your API credentials first!")
    print(f"{Fore.CYAN}Use Ctrl+C to exit the CLI at any time")
    
    # Start the CLI
    cli.run_cli()
    
else:
    print(f"{Fore.RED}Bot not initialized!")
    print(f"{Fore.YELLOW}Please:")
    print(f"1. Set your API credentials in the 'API Credentials Setup' cell")
    print(f"2. Re-run that cell to initialize the bot")
    print(f"3. Then run this cell again")

Starting the Binance Trading Bot CLI...
Make sure you have set your API credentials first!
Use Ctrl+C to exit the CLI at any time

BINANCE FUTURES TESTNET TRADING BOT
Connected to Binance Testnet
This is TESTNET - No real money involved
        

MAIN MENU:
1.  Account Information
2.  Symbol Information
3.  Place Market Order
4.  Place Limit Order
5.  Place Stop-Limit Order
6.  Place OCO Order
7.  View Positions
8.  View Open Orders
9.  Cancel Order
10. Cancel All Orders
11. Close Position
12. Account Balance
0.  Exit
        
 Goodbye! Happy trading!


In [86]:
# Example 1: Get Account Information
if 'bot' in globals() and bot is not None:
    try:
        print(f"{Fore.CYAN}Getting account information...")
        account_data = bot.get_account_info()
        
        print(f"{Fore.GREEN}Account Info Retrieved:")
        print(f"Total Wallet Balance: {account_data['account_info']['totalWalletBalance']} USDT")
        print(f"Available Balance: {account_data['account_info']['availableBalance']} USDT")
        
    except Exception as e:
        print(f"{Fore.RED}Error: {e}")
else:
    print(f"{Fore.YELLOW}Please initialize the bot first")

Getting account information...


2025-10-02 23:41:52,912 - TradingBot - INFO - Account information retrieved successfully


Account Info Retrieved:
Total Wallet Balance: 0.00000000 USDT
Available Balance: 0.00000000 USDT


In [87]:
# Example 2: Get Symbol Information and Current Price
if 'bot' in globals() and bot is not None:
    try:
        symbol = "BTCUSDT"  # You can change this to any symbol
        print(f"{Fore.CYAN}Getting {symbol} information...")
        
        info = bot.get_symbol_info(symbol)
        
        print(f"{Fore.GREEN}{symbol} Info:")
        print(f"Current Price: ${info['current_price']:.2f}")
        print(f"Status: {info['symbol_info']['status']}")
        print(f"Base Asset: {info['symbol_info']['baseAsset']}")
        print(f"Quote Asset: {info['symbol_info']['quoteAsset']}")
        
    except Exception as e:
        print(f"{Fore.RED}Error: {e}")
else:
    print(f"{Fore.YELLOW}Please initialize the bot first")

Getting BTCUSDT information...


2025-10-02 23:41:53,835 - TradingBot - INFO - Retrieved info for BTCUSDT


BTCUSDT Info:
Current Price: $120076.20
Status: TRADING
Base Asset: BTC
Quote Asset: USDT


In [88]:
# Example 3: Test Order Placement (DEMO - Commented Out for Safety)
# WARNING: UNCOMMENT ONLY WHEN YOU'RE READY TO PLACE ACTUAL TESTNET ORDERS

print(f"{Fore.YELLOW}Order Placement Examples (Currently Commented for Safety)")
print(f"{Fore.CYAN}Uncomment the code below when you're ready to test actual orders:")
print()
print("# Example Market Order:")
print("# order = bot.place_market_order('BTCUSDT', 'BUY', 0.001)")
print("# print(f'Market Order ID: {order[\"orderId\"]}')") 
print()
print("# Example Limit Order:")
print("# current_price = bot.get_symbol_info('BTCUSDT')['current_price']")
print("# limit_price = current_price * 0.95  # 5% below current price")
print("# order = bot.place_limit_order('BTCUSDT', 'BUY', 0.001, limit_price)")
print("# print(f'Limit Order ID: {order[\"orderId\"]}')") 
print()
print("# Example Stop-Limit Order:")
print("# current_price = bot.get_symbol_info('BTCUSDT')['current_price']")
print("# stop_price = current_price * 0.95")
print("# limit_price = current_price * 0.94")
print("# order = bot.place_stop_limit_order('BTCUSDT', 'SELL', 0.001, stop_price, limit_price)")
print("# print(f'Stop-Limit Order ID: {order[\"orderId\"]}')") 

print(f"\n{Fore.GREEN}Tip: Use the CLI interface above for interactive order placement!")

Order Placement Examples (Currently Commented for Safety)
Uncomment the code below when you're ready to test actual orders:

# Example Market Order:
# order = bot.place_market_order('BTCUSDT', 'BUY', 0.001)
# print(f'Market Order ID: {order["orderId"]}')

# Example Limit Order:
# current_price = bot.get_symbol_info('BTCUSDT')['current_price']
# limit_price = current_price * 0.95  # 5% below current price
# order = bot.place_limit_order('BTCUSDT', 'BUY', 0.001, limit_price)
# print(f'Limit Order ID: {order["orderId"]}')

# Example Stop-Limit Order:
# current_price = bot.get_symbol_info('BTCUSDT')['current_price']
# stop_price = current_price * 0.95
# limit_price = current_price * 0.94
# order = bot.place_stop_limit_order('BTCUSDT', 'SELL', 0.001, stop_price, limit_price)
# print(f'Stop-Limit Order ID: {order["orderId"]}')

Tip: Use the CLI interface above for interactive order placement!


## Documentation & Features

### Completed Features

**Core Trading Functions:**
- Market Orders (Buy/Sell)
- Limit Orders (Buy/Sell) 
- Stop-Limit Orders
- OCO (One-Cancels-Other) Orders
- Position Management
- Order Cancellation

**Account Management:**
- Account Information & Balance
- Position Tracking
- Open Orders Monitoring
- Real-time PnL Display

**Safety & Logging:**
- Comprehensive Error Handling
- API Request/Response Logging
- Input Validation
- Testnet-Only Operation

**User Interface:**
- Interactive CLI Menu
- Colored Output for Better UX
- Step-by-step Guidance
- Real-time Status Updates

### Advanced Features

**Risk Management:**
- Position size limits
- Daily trade limits
- Automatic stop-loss suggestions

**Order Types:**
- TWAP (Time-Weighted Average Price) - Can be implemented
- Grid Trading - Can be implemented
- Advanced OCO with multiple legs

### Technical Implementation

**Architecture:**
- Object-oriented design for reusability
- Modular structure with clear separation
- Comprehensive logging system
- Type hints for better code maintenance

**API Integration:**
- Official Binance python-binance library
- Testnet environment for safe testing
- Proper error handling for API failures
- Rate limiting compliance

### Usage Instructions

1. **Setup**: Get Binance Testnet credentials
2. **Configure**: Update API keys in the credentials cell
3. **Test**: Run account info and symbol info examples
4. **Trade**: Use the CLI interface for interactive trading
5. **Monitor**: Check positions and orders regularly

### Important Notes

- **TESTNET ONLY**: No real money involved
- **Always Test First**: Verify with small amounts
- **Monitor Positions**: Keep track of open positions
- **Use Stop Losses**: Implement proper risk management
- **Stay Updated**: Check logs for any API issues